<a href="https://colab.research.google.com/github/m-hassaan-ar/ai-lab/blob/main/lab6/q5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [7]:
# task5_sudoku_with_extras.py

from ortools.sat.python import cp_model

def main():
    model = cp_model.CpModel()

    # 9×9 Sudoku variables
    cells = [[model.NewIntVar(1,9,f"c{r}{c}") for c in range(9)] for r in range(9)]

    # example puzzle (0 = empty)
    puzzle = [
        [5,3,0,0,7,0,0,0,0],
        [6,0,0,1,9,5,0,0,0],
        [0,9,8,0,0,0,0,6,0],
        [8,0,0,0,6,0,0,0,3],
        [4,0,0,8,0,3,0,0,1],
        [7,0,0,0,2,0,0,0,6],
        [0,6,0,0,0,0,2,8,0],
        [0,0,0,4,1,9,0,0,5],
        [0,0,0,0,8,0,0,7,9],
    ]

    # apply givens
    for r in range(9):
        for c in range(9):
            if puzzle[r][c]:
                model.Add(cells[r][c] == puzzle[r][c])

    # standard row/col/subgrid constraints
    for i in range(9):
        model.AddAllDifferent(cells[i])                     # row
        model.AddAllDifferent([cells[r][i] for r in range(9)])  # col

    for br in range(3):
        for bc in range(3):
            block = [cells[r][c]
                     for r in range(br*3, br*3+3)
                     for c in range(bc*3, bc*3+3)]
            model.AddAllDifferent(block)

    # diagonal sums divisible by 3
    d1 = [cells[i][i] for i in range(9)]
    d2 = [cells[i][8-i] for i in range(9)]
    s1 = model.NewIntVar(0,81,"s1")
    s2 = model.NewIntVar(0,81,"s2")
    model.Add(s1 == sum(d1))
    model.Add(s2 == sum(d2))
    model.AddModuloEquality(0, s1, 3)
    model.AddModuloEquality(0, s2, 3)

    # no two adjacent primes
    primes = {2,3,5,7}
    for r in range(9):
        for c in range(9):
            for dr,dc in [(1,0),(0,1)]:
                rr, cc = r+dr, c+dc
                if rr < 9 and cc < 9:
                    # if both are prime → disallow
                    b1 = model.NewBoolVar(f"p{r}{c}")
                    b2 = model.NewBoolVar(f"p{rr}{cc}")
                    model.AddAllowedAssignments([cells[r][c], b1],
                                                [(p,1) for p in primes] + [(x,0) for x in range(1,10) if x not in primes])
                    model.AddAllowedAssignments([cells[rr][cc], b2],
                                                [(p,1) for p in primes] + [(x,0) for x in range(1,10) if x not in primes])
                    model.AddBoolOr([b1.Not(), b2.Not()])

    solver = cp_model.CpSolver()
    solver.Solve(model)

    # print solution
    for r in range(9):
        print([solver.Value(cells[r][c]) for c in range(9)])

if __name__ == "__main__":
    main()


[257698037819, 0, 0, 0, 0, 0, 455620904, 0, 0]
[0, 0, 455716786, 132973124493536, 0, 0, 0, 0, 0]
[0, 0, 0, 4564793311245803894, 4564793846201382031, 0, 0, 0, 0]
[0, 0, 0, 3, 49, 132974464339168, 455154560, 455708208, 132974464339168]
[48, 80, 455622328, -6393678766025965013, -9187269055667221662, 36170086419038336, 279172874260, 13, 300647710787]
[0, 0, 1169, 455161744, 132974464340208, 455708160, 455708160, 65, 97]
[455709376, 455707984, 32, 32, 455622600, 0, 0, 33, 455708432]
[132974464339168, 144, 80, 455618280, -57011855297341598, -9208326298114189470, 36170086419038336, 292057776148, 8589934605]
[506806141044, 121, 0, 145, 455710096, 455707744, -9187201950435737472, 296361164928, 68]
